In [ ]:
"""
BatteryMind - Ensemble Model Design and Development Notebook

Comprehensive ensemble model development for battery management systems.
Combines transformer predictions, reinforcement learning decisions, and
physics-based constraints for robust battery optimization.

Features:
- Multi-model ensemble architecture
- Transformer + RL integration
- Physics-based constraint validation
- Adaptive model weighting
- Uncertainty quantification
- Real-time prediction optimization
- Performance benchmarking

Author: BatteryMind Development Team
Version: 1.0.0
"""

# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.ensemble import VotingRegressor, StackingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Import BatteryMind components
import sys
sys.path.append('../../')

from transformers.ensemble_model import EnsembleModel, VotingClassifier, StackingRegressor, ModelFusion
from transformers.battery_health_predictor import BatteryHealthPredictor
from transformers.degradation_forecaster import DegradationForecaster
from transformers.optimization_recommender import OptimizationRecommender
from reinforcement_learning.agents import ChargingAgent
from reinforcement_learning.environments import BatteryEnvironment
from training_data.generators import SyntheticDataGenerator
from evaluation.metrics import accuracy_metrics, performance_metrics
from utils.model_utils import load_model, save_model
from utils.logging_utils import setup_logger

# Configure logging
logger = setup_logger('ensemble_development', 'ensemble_model_design.log')

print("🔋 BatteryMind Ensemble Model Development Environment")
print("=" * 60)

# =============================================================================
# 1. DATA PREPARATION AND LOADING
# =============================================================================

print("\n1. Data Preparation and Loading")
print("-" * 40)

# Generate synthetic training data
data_generator = SyntheticDataGenerator(
    num_batteries=1000,
    simulation_days=30,
    sampling_rate=1.0  # 1 Hz sampling
)

print("📊 Generating synthetic training data...")
synthetic_data = data_generator.generate_fleet_data()
print(f"✓ Generated {len(synthetic_data)} data points")

# Prepare data for different model types
def prepare_ensemble_data(data):
    """Prepare data for different ensemble components."""
    
    # Features for transformer models
    transformer_features = [
        'voltage', 'current', 'temperature', 'soc', 'soh',
        'internal_resistance', 'power_demand', 'ambient_temperature',
        'cycle_count', 'age_days'
    ]
    
    # Features for RL models
    rl_features = [
        'soc', 'temperature', 'voltage', 'current', 'power_demand',
        'soh', 'internal_resistance'
    ]
    
    # Target variables
    targets = {
        'health': 'soh',
        'degradation': 'soh',  # Future SoH prediction
        'optimization': 'power_demand'  # Optimal power recommendation
    }
    
    prepared_data = {
        'transformer_features': data[transformer_features],
        'rl_features': data[rl_features],
        'targets': {k: data[v] for k, v in targets.items()}
    }
    
    return prepared_data

# Prepare data
ensemble_data = prepare_ensemble_data(synthetic_data)

print(f"✓ Data prepared for ensemble models")
print(f"  - Transformer features: {ensemble_data['transformer_features'].shape}")
print(f"  - RL features: {ensemble_data['rl_features'].shape}")
print(f"  - Target variables: {len(ensemble_data['targets'])}")

# Split data for training and validation
from sklearn.model_selection import train_test_split

# Split transformer data
X_transformer = ensemble_data['transformer_features']
y_health = ensemble_data['targets']['health']

X_train_tf, X_val_tf, y_train_health, y_val_health = train_test_split(
    X_transformer, y_health, test_size=0.2, random_state=42
)

# Split RL data
X_rl = ensemble_data['rl_features']
y_optimization = ensemble_data['targets']['optimization']

X_train_rl, X_val_rl, y_train_opt, y_val_opt = train_test_split(
    X_rl, y_optimization, test_size=0.2, random_state=42
)

print(f"✓ Data split completed")
print(f"  - Training samples: {len(X_train_tf)}")
print(f"  - Validation samples: {len(X_val_tf)}")

# =============================================================================
# 2. INDIVIDUAL MODEL DEVELOPMENT
# =============================================================================

print("\n2. Individual Model Development")
print("-" * 40)

# 2.1 Transformer-based Battery Health Predictor
print("🤖 Developing Battery Health Predictor...")

# Simple transformer-like model for demonstration
class BatteryHealthTransformer(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Feature extraction layers
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        
        # Attention mechanism (simplified)
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=4,
            dropout=0.1,
            batch_first=True
        )
        
        # Output layer
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, 1),
            nn.Sigmoid()  # SoH between 0 and 1
        )
    
    def forward(self, x):
        # Feature extraction
        features = self.feature_extractor(x)
        
        # Add sequence dimension for attention
        features = features.unsqueeze(1)
        
        # Apply attention
        attended, _ = self.attention(features, features, features)
        
        # Remove sequence dimension
        attended = attended.squeeze(1)
        
        # Output prediction
        output = self.output_layer(attended)
        
        return output

# Initialize and train transformer model
health_transformer = BatteryHealthTransformer(
    input_size=X_train_tf.shape[1],
    hidden_size=128,
    num_layers=2
)

# Training setup
criterion = nn.MSELoss()
optimizer = optim.Adam(health_transformer.parameters(), lr=0.001)

# Convert to tensors
X_train_tensor = torch.FloatTensor(X_train_tf.values)
y_train_tensor = torch.FloatTensor(y_train_health.values).unsqueeze(1)
X_val_tensor = torch.FloatTensor(X_val_tf.values)
y_val_tensor = torch.FloatTensor(y_val_health.values).unsqueeze(1)

# Training loop
print("  📈 Training transformer model...")
transformer_losses = []
val_losses = []

for epoch in range(50):
    # Training
    health_transformer.train()
    optimizer.zero_grad()
    outputs = health_transformer(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    transformer_losses.append(loss.item())
    
    # Validation
    health_transformer.eval()
    with torch.no_grad():
        val_outputs = health_transformer(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_losses.append(val_loss.item())
    
    if epoch % 10 == 0:
        print(f"    Epoch {epoch}: Train Loss = {loss.item():.4f}, Val Loss = {val_loss.item():.4f}")

print("✓ Transformer model trained")

# 2.2 Physics-based Model
print("🔬 Developing Physics-based Model...")

class PhysicsBasedModel:
    """Physics-based battery model for SoH prediction."""
    
    def __init__(self):
        self.model_params = {
            'calendar_aging_factor': 0.001,
            'cycle_aging_factor': 0.0001,
            'temperature_factor': 0.01,
            'capacity_fade_rate': 0.02
        }
    
    def predict(self, X):
        """Predict SoH based on physics equations."""
        predictions = []
        
        for _, row in X.iterrows():
            # Extract relevant features
            temperature = row.get('temperature', 25.0)
            cycle_count = row.get('cycle_count', 0)
            age_days = row.get('age_days', 0)
            current_load = row.get('current', 0)
            
            # Calendar aging
            calendar_aging = (self.model_params['calendar_aging_factor'] * 
                            age_days * np.exp(0.1 * (temperature - 25)))
            
            # Cycle aging
            cycle_aging = (self.model_params['cycle_aging_factor'] * 
                          cycle_count * (1 + abs(current_load) / 100))
            
            # Temperature effects
            temp_effect = self.model_params['temperature_factor'] * max(0, temperature - 40)
            
            # Combine aging effects
            total_aging = calendar_aging + cycle_aging + temp_effect
            
            # Calculate SoH
            soh = max(0.5, 1.0 - total_aging)
            predictions.append(soh)
        
        return np.array(predictions)

physics_model = PhysicsBasedModel()

# Test physics model
physics_predictions = physics_model.predict(X_val_tf)
physics_mae = mean_absolute_error(y_val_health, physics_predictions)
print(f"✓ Physics model MAE: {physics_mae:.4f}")

# 2.3 Traditional ML Models
print("📊 Developing Traditional ML Models...")

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

# Initialize traditional models
traditional_models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression(),
    'SVR': SVR(kernel='rbf', C=1.0)
}

# Train traditional models
traditional_predictions = {}
traditional_scores = {}

for name, model in traditional_models.items():
    print(f"  🔧 Training {name}...")
    model.fit(X_train_tf, y_train_health)
    
    # Predictions
    predictions = model.predict(X_val_tf)
    traditional_predictions[name] = predictions
    
    # Scores
    mae = mean_absolute_error(y_val_health, predictions)
    mse = mean_squared_error(y_val_health, predictions)
    r2 = r2_score(y_val_health, predictions)
    
    traditional_scores[name] = {
        'MAE': mae,
        'MSE': mse,
        'R2': r2
    }
    
    print(f"    ✓ {name} - MAE: {mae:.4f}, R2: {r2:.4f}")

# =============================================================================
# 3. ENSEMBLE ARCHITECTURE DESIGN
# =============================================================================

print("\n3. Ensemble Architecture Design")
print("-" * 40)

# 3.1 Voting Ensemble
print("🗳️ Creating Voting Ensemble...")

class VotingEnsemble:
    """Voting ensemble combining multiple models."""
    
    def __init__(self, models, weights=None):
        self.models = models
        self.weights = weights or [1/len(models)] * len(models)
        self.model_names = list(models.keys())
    
    def predict(self, X):
        """Make predictions using weighted voting."""
        predictions = []
        
        for name, model in self.models.items():
            if name == 'Transformer':
                # Handle transformer model
                X_tensor = torch.FloatTensor(X.values)
                model.eval()
                with torch.no_grad():
                    pred = model(X_tensor).numpy().flatten()
            elif name == 'Physics':
                pred = model.predict(X)
            else:
                pred = model.predict(X)
            
            predictions.append(pred)
        
        # Weighted average
        predictions = np.array(predictions)
        weighted_pred = np.average(predictions, axis=0, weights=self.weights)
        
        return weighted_pred
    
    def get_individual_predictions(self, X):
        """Get predictions from each model."""
        individual_preds = {}
        
        for name, model in self.models.items():
            if name == 'Transformer':
                X_tensor = torch.FloatTensor(X.values)
                model.eval()
                with torch.no_grad():
                    pred = model(X_tensor).numpy().flatten()
            elif name == 'Physics':
                pred = model.predict(X)
            else:
                pred = model.predict(X)
            
            individual_preds[name] = pred
        
        return individual_preds

# Create voting ensemble
voting_models = {
    'Transformer': health_transformer,
    'Physics': physics_model,
    'Random Forest': traditional_models['Random Forest'],
    'Gradient Boosting': traditional_models['Gradient Boosting']
}

voting_ensemble = VotingEnsemble(voting_models)

# Test voting ensemble
voting_predictions = voting_ensemble.predict(X_val_tf)
voting_mae = mean_absolute_error(y_val_health, voting_predictions)
voting_r2 = r2_score(y_val_health, voting_predictions)

print(f"✓ Voting Ensemble - MAE: {voting_mae:.4f}, R2: {voting_r2:.4f}")

# 3.2 Stacking Ensemble
print("🥞 Creating Stacking Ensemble...")

class StackingEnsemble:
    """Stacking ensemble with meta-learner."""
    
    def __init__(self, base_models, meta_model=None):
        self.base_models = base_models
        self.meta_model = meta_model or LinearRegression()
        self.is_fitted = False
    
    def fit(self, X, y):
        """Fit base models and meta-learner."""
        # Fit base models
        for name, model in self.base_models.items():
            if name not in ['Transformer', 'Physics']:
                model.fit(X, y)
        
        # Generate base predictions for meta-learner
        base_predictions = self._get_base_predictions(X)
        
        # Fit meta-learner
        self.meta_model.fit(base_predictions, y)
        self.is_fitted = True
    
    def _get_base_predictions(self, X):
        """Get predictions from base models."""
        predictions = []
        
        for name, model in self.base_models.items():
            if name == 'Transformer':
                X_tensor = torch.FloatTensor(X.values)
                model.eval()
                with torch.no_grad():
                    pred = model(X_tensor).numpy().flatten()
            elif name == 'Physics':
                pred = model.predict(X)
            else:
                pred = model.predict(X)
            
            predictions.append(pred)
        
        return np.column_stack(predictions)
    
    def predict(self, X):
        """Make predictions using stacking."""
        if not self.is_fitted:
            raise ValueError("Ensemble must be fitted before prediction")
        
        base_predictions = self._get_base_predictions(X)
        return self.meta_model.predict(base_predictions)

# Create and train stacking ensemble
stacking_ensemble = StackingEnsemble(voting_models)
stacking_ensemble.fit(X_train_tf, y_train_health)

# Test stacking ensemble
stacking_predictions = stacking_ensemble.predict(X_val_tf)
stacking_mae = mean_absolute_error(y_val_health, stacking_predictions)
stacking_r2 = r2_score(y_val_health, stacking_predictions)

print(f"✓ Stacking Ensemble - MAE: {stacking_mae:.4f}, R2: {stacking_r2:.4f}")

# 3.3 Adaptive Ensemble with Uncertainty Quantification
print("🎯 Creating Adaptive Ensemble...")

class AdaptiveEnsemble:
    """Adaptive ensemble with dynamic weighting and uncertainty quantification."""
    
    def __init__(self, models, adaptation_rate=0.1):
        self.models = models
        self.adaptation_rate = adaptation_rate
        self.weights = np.ones(len(models)) / len(models)
        self.model_names = list(models.keys())
        self.performance_history = []
    
    def predict_with_uncertainty(self, X):
        """Make predictions with uncertainty estimates."""
        predictions = []
        
        # Get individual predictions
        for name, model in self.models.items():
            if name == 'Transformer':
                X_tensor = torch.FloatTensor(X.values)
                model.eval()
                with torch.no_grad():
                    pred = model(X_tensor).numpy().flatten()
            elif name == 'Physics':
                pred = model.predict(X)
            else:
                pred = model.predict(X)
            
            predictions.append(pred)
        
        predictions = np.array(predictions)
        
        # Weighted prediction
        ensemble_pred = np.average(predictions, axis=0, weights=self.weights)
        
        # Uncertainty as weighted standard deviation
        uncertainty = np.sqrt(np.average((predictions - ensemble_pred)**2, axis=0, weights=self.weights))
        
        return ensemble_pred, uncertainty
    
    def update_weights(self, X, y_true):
        """Update model weights based on recent performance."""
        # Get individual predictions
        individual_preds = {}
        for name, model in self.models.items():
            if name == 'Transformer':
                X_tensor = torch.FloatTensor(X.values)
                model.eval()
                with torch.no_grad():
                    pred = model(X_tensor).numpy().flatten()
            elif name == 'Physics':
                pred = model.predict(X)
            else:
                pred = model.predict(X)
            
            individual_preds[name] = pred
        
        # Calculate individual errors
        errors = []
        for name in self.model_names:
            error = mean_absolute_error(y_true, individual_preds[name])
            errors.append(error)
        
        # Update weights (lower error = higher weight)
        errors = np.array(errors)
        inv_errors = 1 / (errors + 1e-8)  # Avoid division by zero
        new_weights = inv_errors / np.sum(inv_errors)
        
        # Smooth weight updates
        self.weights = (1 - self.adaptation_rate) * self.weights + self.adaptation_rate * new_weights
        
        # Store performance history
        self.performance_history.append({
            'errors': errors,
            'weights': self.weights.copy()
        })

# Create adaptive ensemble
adaptive_ensemble = AdaptiveEnsemble(voting_models)

# Test adaptive ensemble
adaptive_predictions, uncertainty = adaptive_ensemble.predict_with_uncertainty(X_val_tf)
adaptive_mae = mean_absolute_error(y_val_health, adaptive_predictions)
adaptive_r2 = r2_score(y_val_health, adaptive_predictions)

print(f"✓ Adaptive Ensemble - MAE: {adaptive_mae:.4f}, R2: {adaptive_r2:.4f}")

# Update weights and test again
adaptive_ensemble.update_weights(X_val_tf, y_val_health)
adaptive_predictions_updated, uncertainty_updated = adaptive_ensemble.predict_with_uncertainty(X_val_tf)
adaptive_mae_updated = mean_absolute_error(y_val_health, adaptive_predictions_updated)

print(f"✓ Adaptive Ensemble (Updated) - MAE: {adaptive_mae_updated:.4f}")

# =============================================================================
# 4. PERFORMANCE EVALUATION AND COMPARISON
# =============================================================================

print("\n4. Performance Evaluation and Comparison")
print("-" * 40)

# Compile all ensemble results
ensemble_results = {
    'Voting': {
        'predictions': voting_predictions,
        'mae': voting_mae,
        'r2': voting_r2
    },
    'Stacking': {
        'predictions': stacking_predictions,
        'mae': stacking_mae,
        'r2': stacking_r2
    },
    'Adaptive': {
        'predictions': adaptive_predictions,
        'mae': adaptive_mae,
        'r2': adaptive_r2
    },
    'Adaptive (Updated)': {
        'predictions': adaptive_predictions_updated,
        'mae': adaptive_mae_updated,
        'r2': r2_score(y_val_health, adaptive_predictions_updated)
    }
}

# Add individual model results for comparison
for name, scores in traditional_scores.items():
    ensemble_results[name] = {
        'predictions': traditional_predictions[name],
        'mae': scores['MAE'],
        'r2': scores['R2']
    }

# Add physics model
ensemble_results['Physics'] = {
    'predictions': physics_predictions,
    'mae': physics_mae,
    'r2': r2_score(y_val_health, physics_predictions)
}

# Create performance comparison plots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Ensemble Model Performance Comparison', fontsize=16)

# Plot 1: MAE Comparison
ax1 = axes[0, 0]
models = list(ensemble_results.keys())
mae_values = [ensemble_results[m]['mae'] for m in models]

bars = ax1.bar(models, mae_values, color=plt.cm.viridis(np.linspace(0, 1, len(models))))
ax1.set_ylabel('Mean Absolute Error')
ax1.set_title('MAE Comparison')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, mae_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.001,
             f'{value:.4f}', ha='center', va='bottom', fontsize=8)

# Plot 2: R² Comparison
ax2 = axes[0, 1]
r2_values = [ensemble_results[m]['r2'] for m in models]

bars = ax2.bar(models, r2_values, color=plt.cm.plasma(np.linspace(0, 1, len(models))))
ax2.set_ylabel('R² Score')
ax2.set_title('R² Comparison')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

# Add value labels on bars
# Add value labels on bars
for bar, value in zip(bars, r2_values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{value:.4f}', ha='center', va='bottom', fontsize=8)

# Plot 3: Prediction vs Actual for Best Model
ax3 = axes[1, 0]
best_model = max(models, key=lambda m: ensemble_results[m]['r2'])
best_predictions = ensemble_results[best_model]['predictions']

ax3.scatter(y_test, best_predictions, alpha=0.6, color='blue', s=20)
ax3.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax3.set_xlabel('Actual SoH')
ax3.set_ylabel('Predicted SoH')
ax3.set_title(f'Best Model: {best_model}')
ax3.grid(True, alpha=0.3)

# Add R² score on plot
ax3.text(0.05, 0.95, f'R² = {ensemble_results[best_model]["r2"]:.4f}', 
         transform=ax3.transAxes, bbox=dict(boxstyle="round", facecolor='white', alpha=0.8))

# Plot 4: Residuals for Best Model
ax4 = axes[1, 1]
residuals = y_test - best_predictions

ax4.scatter(best_predictions, residuals, alpha=0.6, color='red', s=20)
ax4.axhline(y=0, color='black', linestyle='--', alpha=0.8)
ax4.set_xlabel('Predicted SoH')
ax4.set_ylabel('Residuals')
ax4.set_title(f'Residuals: {best_model}')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed results
print("\n" + "="*60)
print("DETAILED ENSEMBLE MODEL RESULTS")
print("="*60)

for model_name, results in ensemble_results.items():
    print(f"\n{model_name.upper()}:")
    print(f"  MAE: {results['mae']:.6f}")
    print(f"  MSE: {results['mse']:.6f}")
    print(f"  R²:  {results['r2']:.6f}")
    print(f"  Training Time: {results['training_time']:.2f} seconds")

# Identify best performing model
best_model_name = max(ensemble_results.keys(), key=lambda m: ensemble_results[m]['r2'])
print(f"\nBEST PERFORMING MODEL: {best_model_name}")
print(f"Best R² Score: {ensemble_results[best_model_name]['r2']:.6f}")

# Feature importance analysis for ensemble models
print("\n" + "="*60)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Get feature importances where available
feature_names = X_train.columns if hasattr(X_train, 'columns') else [f'feature_{i}' for i in range(X_train.shape[1])]

for model_name, model in ensemble_models.items():
    if hasattr(model, 'feature_importances_'):
        print(f"\n{model_name.upper()} Feature Importances:")
        importances = model.feature_importances_
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        print(feature_importance_df.head(10).to_string(index=False))
    elif hasattr(model, 'coef_'):
        print(f"\n{model_name.upper()} Coefficients:")
        coefficients = np.abs(model.coef_[0]) if len(model.coef_.shape) > 1 else np.abs(model.coef_)
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'coefficient': coefficients
        }).sort_values('coefficient', ascending=False)
        
        print(feature_importance_df.head(10).to_string(index=False))

# Cross-validation analysis
print("\n" + "="*60)
print("CROSS-VALIDATION ANALYSIS")
print("="*60)

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# Define custom scorer
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

cv_results = {}
for model_name, model in ensemble_models.items():
    print(f"\nPerforming 5-fold CV for {model_name}...")
    
    # MAE cross-validation
    mae_scores = cross_val_score(model, X_train, y_train, cv=5, scoring=mae_scorer)
    mae_scores = -mae_scores  # Convert back to positive
    
    # R² cross-validation
    r2_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    
    cv_results[model_name] = {
        'mae_mean': mae_scores.mean(),
        'mae_std': mae_scores.std(),
        'r2_mean': r2_scores.mean(),
        'r2_std': r2_scores.std()
    }
    
    print(f"  MAE: {mae_scores.mean():.4f} ± {mae_scores.std():.4f}")
    print(f"  R²:  {r2_scores.mean():.4f} ± {r2_scores.std():.4f}")

# Cross-validation visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# MAE cross-validation results
models = list(cv_results.keys())
mae_means = [cv_results[m]['mae_mean'] for m in models]
mae_stds = [cv_results[m]['mae_std'] for m in models]

ax1.bar(models, mae_means, yerr=mae_stds, capsize=5, color='skyblue', alpha=0.7)
ax1.set_ylabel('MAE')
ax1.set_title('Cross-Validation MAE Results')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# R² cross-validation results
r2_means = [cv_results[m]['r2_mean'] for m in models]
r2_stds = [cv_results[m]['r2_std'] for m in models]

ax2.bar(models, r2_means, yerr=r2_stds, capsize=5, color='lightcoral', alpha=0.7)
ax2.set_ylabel('R² Score')
ax2.set_title('Cross-Validation R² Results')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Model serialization for deployment
import joblib
import pickle
from datetime import datetime

# Create model metadata
model_metadata = {
    'model_name': production_model_name,
    'model_type': 'ensemble_battery_health',
    'version': '1.0.0',
    'created_date': datetime.now().isoformat(),
    'performance_metrics': {
        'test_mae': float(comparison_df.iloc[0]['Test_MAE']),
        'test_r2': float(comparison_df.iloc[0]['Test_R2']),
        'cv_mae_mean': float(comparison_df.iloc[0]['CV_MAE_Mean']),
        'cv_mae_std': float(comparison_df.iloc[0]['CV_MAE_Std']),
        'cv_r2_mean': float(comparison_df.iloc[0]['CV_R2_Mean']),
        'cv_r2_std': float(comparison_df.iloc[0]['CV_R2_Std'])
    },
    'training_data_shape': X_train.shape,
    'feature_names': feature_names,
    'model_parameters': production_model.get_params() if hasattr(production_model, 'get_params') else 'N/A'
}

# Save production model
model_output_dir = Path('../../model-artifacts/trained_models/ensemble_v1.0')
model_output_dir.mkdir(parents=True, exist_ok=True)

# Save model
joblib.dump(production_model, model_output_dir / 'ensemble_model.pkl')
print(f"✅ Production model saved to: {model_output_dir / 'ensemble_model.pkl'}")

# Save metadata
with open(model_output_dir / 'model_metadata.json', 'w') as f:
    json.dump(model_metadata, f, indent=2)
print(f"✅ Model metadata saved to: {model_output_dir / 'model_metadata.json'}")

# Save training history
training_history = {
    'all_models_results': ensemble_results,
    'cross_validation_results': cv_results,
    'model_comparison': comparison_df.to_dict('records')
}

with open(model_output_dir / 'training_history.json', 'w') as f:
    json.dump(training_history, f, indent=2, default=str)
print(f"✅ Training history saved to: {model_output_dir / 'training_history.json'}")

print("\n" + "="*70)
print("FINAL RECOMMENDATIONS AND NEXT STEPS")
print("="*70)

print(f"""
🎯 PRODUCTION MODEL SELECTION:
   Selected Model: {production_model_name}
   Performance: R² = {comparison_df.iloc[0]['Test_R2']:.4f}, MAE = {comparison_df.iloc[0]['Test_MAE']:.4f}
   
🚀 NEXT STEPS FOR DEPLOYMENT:
   1. Model Validation: Test on additional holdout datasets
   2. A/B Testing: Compare against current production model
   3. Integration: Integrate with BatteryMind inference pipeline
   4. Monitoring: Set up model performance monitoring
   5. Retraining: Establish automated retraining pipeline
   
📊 MODEL IMPROVEMENTS:
   1. Feature Engineering: Add more temporal features
   2. Hyperparameter Tuning: Use Optuna for optimization
   3. Ensemble Diversity: Include neural network models
   4. Cross-Chemistry: Train separate models for different chemistries
   
⚠️  CONSIDERATIONS:
   1. Model interpretability for regulatory compliance
   2. Computational efficiency for real-time inference
   3. Data drift monitoring and adaptation
   4. Safety constraints integration
""")

print("\n🎉 ENSEMBLE MODEL DEVELOPMENT COMPLETED SUCCESSFULLY!")
print("="*70)
